## Install and import depdencies 

In [ ]:
!conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

In [ ]:
!conda install ultralytics
!pip install --upgrade ultralytics

In [1]:
from ultralytics import YOLO
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt

## Prep training materials

In [2]:
import os
import shutil
import random

Please make sure all images are labeled using YOLO format.

INSTRUCTIONS:
1. Create a folder named "data" OUTSIDE of the local repo. This folder will contain all your data.
2. Create a subdirectory inside the data folder and add one folder named "unsplit". Add your images and labels folders to this subdirectory.
   I have included an exmaple directory inside the RSYOLOV8 folder. This is how your data folder should be set up before running the next part of the code.
3. Please go into the config.yaml file and set the "path" variable to the ABSOLUTE PATH of the data folder.
   

In [3]:
# Set the path to the data folder below as well.
data_dir = "D:/RecieptScanner\data"

In [4]:
# Define paths
unsplit_dir = os.path.join(data_dir, 'unsplit')
images_dir = os.path.join(unsplit_dir, 'images')
labels_dir = os.path.join(unsplit_dir, 'labels')

In [5]:
# New directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [6]:
# Create new directories
for split_dir in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(split_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(split_dir, 'labels'), exist_ok=True)

In [7]:
train_ratio = 0.85
val_ratio = 0.1
test_ratio = 0.05

In [8]:
# get unsplit images and labels
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
labels = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

In [9]:
# Ensure there is a label for each image
images = [img for img in images if f'{os.path.splitext(img)[0]}.txt' in labels]

In [10]:
random.shuffle(images)

In [11]:
# Calculate split sizes
total_images = len(images)
train_size = int(total_images * train_ratio)
val_size = int(total_images * val_ratio)
test_size = total_images - train_size - val_size

In [12]:
# Split images and labels
train_images = images[:train_size]
val_images = images[train_size:train_size + val_size]
test_images = images[train_size + val_size:]

In [13]:
# Function to move files to appropriate directories
def move_files(file_list, source_img_dir, source_lbl_dir, target_dir):
    for file_name in file_list:
        img_source_path = os.path.join(source_img_dir, file_name)
        lbl_source_path = os.path.join(source_lbl_dir, f'{os.path.splitext(file_name)[0]}.txt')
        
        img_target_path = os.path.join(target_dir, 'images', file_name)
        lbl_target_path = os.path.join(target_dir, 'labels', f'{os.path.splitext(file_name)[0]}.txt')
        
        shutil.copy(img_source_path, img_target_path)
        shutil.copy(lbl_source_path, lbl_target_path)

In [14]:
# Move files to train, val, test directories
move_files(train_images, images_dir, labels_dir, train_dir)
move_files(val_images, images_dir, labels_dir, val_dir)
move_files(test_images, images_dir, labels_dir, test_dir)

# You are now ready to train!

## Select model architecture

In [15]:
model = YOLO('yolov8n.yaml')  # 'yolov8n.yaml' for untrained or 'yolov8s.pt' for pretrained weights

## Train the model

In [16]:
# data: Specifies the path to the dataset configuration file
# imgsz: Sets the image size
# batch: Sets the batch size
# epochs: Sets the number of epochs
# workers: Sets the number of data loader workers ( i have no idea what this does )
#model.train(data='config.yaml', workers=2, iou=0.5, 
#    epochs=1000, batch= 16, imgsz=640, optimizer='SGD', weight_decay=0.0005, momentum=0.9,)
model.train(data="config.yaml", epochs=500, workers=2)
# model.export(format="onnx")

New https://pypi.org/project/ultralytics/8.2.79 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=config.yaml, epochs=500, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning D:\RecieptScanner\data\train\labels.cache... 165 images, 0 backgrounds, 0 corrupt: 100%|██████████| 165
val: Scanning D:\RecieptScanner\data\val\labels... 107 images, 0 backgrounds, 0 corrupt: 100%|██████████| 107/107 [00:0

val: New cache created: D:\RecieptScanner\data\val\labels.cache


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs\detect\train7
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/500      2.46G      6.613      6.276      4.331         76        640: 100%|██████████| 11/11 [00:08<00:00,  1.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<0

...

                   all        107       1009      0.829      0.857      0.919      0.616



500 epochs completed in 0.753 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 6.3MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics YOLOv8.2.67  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
YOLOv8n summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<0


                   all        107       1009      0.853      0.845      0.919      0.618
                  item        103        807      0.807      0.867      0.918      0.614
              subtotal         97         97      0.897       0.81      0.901      0.594
                 total        105        105      0.855      0.857      0.938      0.645
Speed: 0.3ms preprocess, 3.9ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs\detect\train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026CF06D2C10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

## Export as tflite

In [ ]:

model = YOLO('trained models/best.pt')

model.export(format="onnx")

In [ ]:
model2 = YOLO('yolov8s.pt')
model2.train(data="config.yaml", epochs=2000, workers=2)
model2.export(format="onnx")

## Testing

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

In [ ]:
# absolute path
%matplotlib inline
model = YOLO('D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/bestonnx.pt')
img = cv2.imread('D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/images/reciept.jpg')
results = model(img, conf=0.4, iou=0.4)

for result in results:
    boxes = result.boxes.cpu().numpy()
    
    # convert into cv2 rectangle
    for xyxy in boxes.xyxy:
        print(xyxy, "hellO")
annotated_img = results[0].plot()
# Save the annotated image
if os.path.exists('annotated_image.jpg'):
        os.remove('annotated_image.jpg')
cv2.imwrite('annotated_image.jpg', results[0].plot())

# Load and display the saved image using matplotlib
plt.figure()
annotated_img = cv2.imread('annotated_image.jpg')
plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
plt.title("Annotated Image with Detections")
plt.show()

## Onnx test

In [ ]:
!pip install onnxruntime

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import tensorflow as tf

# Running prediction on ONNX using method 1 (javascript similarity)

def preprocess_image(image_path, target_size):
    image = Image.open(image_path).convert('RGB')
    image_resized = image.resize(target_size, Image.LANCZOS)
    image_array = np.array(image_resized) / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.transpose(image_array, (0, 3, 1, 2))
    return image_array.astype(np.float32), image

def run_onnx_model(model_path, image_array):
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    result = session.run([output_name], {input_name: image_array})
    return result

def onnx_nms(output, image_size):
    num_detections = 8400
    output_tensor = output.flatten()
    output_array = output_tensor.tolist()

    tf_boxes = []
    scores = []
    classes = []

    for i in range(num_detections):
        offset = i * 7
        classes_scores = output_array[offset + 4: offset + 7]
        max_score = max(classes_scores)
        max_class_index = classes_scores.index(max_score)

        if max_score >= 0.25:
            box = [
                output_array[offset] - 0.5 * output_array[offset + 2], # x
                output_array[offset + 1] - 0.5 * output_array[offset + 3], # y
                output_array[offset + 2], # width
                output_array[offset + 3], # height
            ]

            tf_boxes.append(box)
            scores.append(max_score)
            classes.append(max_class_index)

    max_output_size = 100
    iou_threshold = 0.1
    score_threshold = 400.0

    tf_boxes_tensor = tf.convert_to_tensor(tf_boxes)
    scores_tensor = tf.convert_to_tensor(scores)
    
    selected_indices = tf.image.non_max_suppression(
        tf_boxes_tensor, scores_tensor, max_output_size, iou_threshold, score_threshold
    )

    nms_boxes = []
    for index in selected_indices.numpy():
        nms_boxes.append({
            'bbox': tf_boxes[index],
            'score': scores[index],
            'class': classes[index]
        })
    print(nms_boxes)
    print(len(nms_boxes))
    return nms_boxes

def draw_bounding_boxes(image, bounding_boxes):
    draw = ImageDraw.Draw(image)
    for bbox in bounding_boxes:
        box = bbox['bbox']
        draw.rectangle([(box[0], box[1]), (box[0] + box[2], box[1] + box[3])], outline="red", width=2)
        draw.text((box[0], box[1]), f"Class: {bbox['class']} Score: {bbox['score']:.2f}", fill="red")
    return image

def main(image_path, model_path, target_size):
    preprocessed_image, original_image = preprocess_image(image_path, target_size)
    model_output = run_onnx_model(model_path, preprocessed_image)
    detection_results = onnx_nms(model_output[0], original_image.size)

    image_with_boxes = draw_bounding_boxes(original_image, detection_results)
    plt.imshow(image_with_boxes)
    plt.axis('off')
    plt.show()

# Example usage
image_path = 'D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/images/reciept.jpg'
model_path = 'D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/bestonnx.onnx'
target_size = (640, 640)

main(image_path, model_path, target_size)

In [ ]:
import numpy as np
import onnxruntime as ort
from PIL import Image, ImageDraw
import cv2

# GPT Procedure

# Function to perform Non-Maximum Suppression
def non_max_suppression(prediction, conf_thresh=0.4, iou_thresh=0.1):
    boxes = prediction[:, :4]  # Extract bounding boxes
    scores = prediction[:, 4] * prediction[:, 5]  # Compute scores
    classes = prediction[:, 6]  # Extract class ids

    # Convert boxes to corner coordinates
    boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] + boxes[:, 1]

    # Apply score threshold
    indices = scores > conf_thresh
    boxes, scores, classes = boxes[indices], scores[indices], classes[indices]

    # Apply NMS
    indices = cv2.dnn.NMSBoxes(boxes.tolist(), scores.tolist(), conf_thresh, iou_thresh)
    if len(indices) > 0:
        indices = indices.flatten()
        boxes, scores, classes = boxes[indices], scores[indices], classes[indices]

    return boxes, scores, classes

# Function to draw bounding boxes on an image
def draw_bounding_boxes(image, bounding_boxes):
    draw = ImageDraw.Draw(image)
    for bbox in bounding_boxes:
        box = bbox['bbox']
        draw.rectangle([(box[0], box[1]), (box[0] + box[2], box[1] + box[3])], outline="red", width=2)
        draw.text((box[0], box[1]), f"Class: {bbox['class']} Score: {bbox['score']:.2f}", fill="red")
    return image

# Main function to run inference
def main(image_path, model_path, target_size):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    image_resized = image.resize(target_size)
    image_np = np.array(image_resized).astype(np.float32)
    image_np = np.transpose(image_np, (2, 0, 1))  # Change to (C, H, W) format
    image_np = np.expand_dims(image_np, axis=0)  # Add batch dimension
    image_np /= 255.0  # Normalize to [0, 1]

    # Load the ONNX model
    ort_session = ort.InferenceSession(model_path)
    
    # Run inference
    outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: image_np})

    # Apply NMS
    prediction = outputs[0].reshape(-1, 7)  # Reshape to (8400, 7)
    boxes, scores, classes = non_max_suppression(prediction)

    # Prepare bounding boxes for drawing
    bounding_boxes = [{'bbox': [box[0], box[1], box[2] - box[0], box[3] - box[1]], 'class': int(cls), 'score': score}
                      for box, cls, score in zip(boxes, classes, scores)]
    
    # Draw bounding boxes on the image
    result_image = draw_bounding_boxes(image, bounding_boxes)
    
    return result_image

# Example usage
image_path = 'D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/images/reciept.jpg'
model_path = 'D:/RecieptScanner/reciept-scanner/RSYOLOV8/testModels/bestonnx.onnx'
target_size = (640, 640)

result_image = main(image_path, model_path, target_size)
result_image.show()
